In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.1")

Load documents

In [2]:
from langchain_community.document_loaders import PyPDFLoader

file_path = ("/opt/cloudadm/llm_rag/978-3-319-63588-0.pdf")
loader = PyPDFLoader(file_path)
docs = loader.load()

Split

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "dunzhang/stella_en_1.5B_v5"
model_kwargs = {'device': 'cuda',
                'trust_remote_code': True}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [5]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from uuid import uuid4

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vectorstore = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

uuids = [str(uuid4()) for _ in range(len(docs))]

vectorstore.add_documents(documents=docs, ids=uuids)

retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))
# Run
question = "What are the Rules for Propositional Connectives?"
questions = generate_queries_decomposition.invoke({"question":question})

# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

q_a_pairs = ""
for q in questions:
    print('q a pair generation iteration')
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [6]:
questions

NameError: name 'questions' is not defined

Retrieval + Generation

In [5]:
from langchain import hub
from langchain_core.runnables import RunnableLambda
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import OllamaEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from uuid import uuid4

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vectorstore = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

uuids = [str(uuid4()) for _ in range(len(docs))]

vectorstore.add_documents(documents=docs, ids=uuids)

retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def inspect(state):
    """Print the state passed between Runnables in a langchain and pass it on"""
    print(state)
    return state

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | RunnableLambda(inspect)
    | prompt
    | RunnableLambda(inspect)
    | llm
    | StrOutputParser()
)

# Question

/opt/cloudadm/llm_rag/.venv/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Decomposition

In [6]:
rag_chain.invoke("What are the Rules for Propositional Connectives?")

{'context': '6.2 Truth and Proof 181\nRules for Propositional Connectives\nProof rule ^L is for handling conjunctions (P ^Q) as one of the assumptions in the\nantecedent on the left of the sequent turnstile (`). Assuming the conjunction P ^Q\nis the same as assuming each conjunct P as well as Q separately.\n^L G ;P;Q `D\nG ;P ^Q `D\nRule ^Lexpresses that if a conjunction P ^Q is among the list of available assump-\ntions in the antecedent, then we might just as well assume both conjuncts ( P and\nQ, respectively) separately. Assuming a conjunction P ^Q is the same as assuming\nboth conjuncts P and Q. So, if we set out to prove a sequent of the form in the\nconclusion (G ;P ^Q `D), then we can justify this sequent by instead proving the\nsequent in the corresponding premise (G ;P;Q `D), where the only difference is that\nthe two assumptions P and Q are now assumed separately in the premise rather than\njointly as a single conjunction, as in the conclusion.\nIf we keep on using proof rul

'The rules for propositional connectives include ^L, ^R, _L, _R, !R, and !L, which handle conjunctions, disjunctions, and implications in the antecedent or succedent. Each rule captures the meaning of a specific logical operator, such as assuming a conjunction being equivalent to assuming each conjunct separately. The rules provide a systematic way to break down complex formulas into simpler ones, allowing for more efficient proofs.'